In [ ]:
import pandas as pd
import tensorflow as tf
import collections,numpy
from pandas import DataFrame
import warnings
import numpy as np # used for handling numbers
from keras import backend as K
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler # used for encoding categorical data
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.preprocessing import StandardScaler # used for feature scaling
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt # plotting
from math import * # sqrt() etc
from keras.utils import np_utils
from keras.utils import to_categorical
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import collections
from functools import reduce
import operator
import time


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [ ]:
!pip install mxnet-cu100

     |████████████████████████████████| 540.1MB 29kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
import mxnet as mx
# Testing that GPU works.
a = mx.nd.ones((2, 3), mx.gpu())
b = a * 2 + 1
print(b)


[[3. 3. 3.]
 [3. 3. 3.]]
<NDArray 2x3 @gpu(0)>


In [ ]:
import mxnet as mx
from mxnet import nd, autograd      

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


DATA EXTRACTION

In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/covtype.csv')

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
Elevation                             581012 non-null int64
Aspect                                581012 non-null int64
Slope                                 581012 non-null int64
Horizontal_Distance_To_Hydrology      581012 non-null int64
Vertical_Distance_To_Hydrology        581012 non-null int64
Horizontal_Distance_To_Roadways       581012 non-null int64
Hillshade_9am                         581012 non-null int64
Hillshade_Noon                        581012 non-null int64
Hillshade_3pm                         581012 non-null int64
Horizontal_Distance_To_Fire_Points    581012 non-null int64
Wilderness_Area1                      581012 non-null int64
Wilderness_Area2                      581012 non-null int64
Wilderness_Area3                      581012 non-null int64
Wilderness_Area4                      581012 non-null int64
Soil_Type1                            581012 non-

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.005890,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,0.497379,0.220882,0.495897,0.244087,0.072039,0.113066,0.090731,0.144499,0.052356,0.105775,0.013442,0.017550,0.044387,0.230245,0.144579,0.221186,0.170590,0.032092,0.002272,0.069804,0.076518,0.057077,0.082902,0.125228,0.037950,0.232681,0.299197,0.187833,0.028551,0.066605,0.043193,0.040318,0.398762,0.221879,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [ ]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


DATA PREPROCESSING

In [ ]:
df=df.drop(['Soil_Type7','Soil_Type8','Soil_Type15','Soil_Type36'],axis=1)
print (df.shape)

(581012, 51)


In [ ]:
def cut_down_col(df):
  cols=df.columns
  r,c=df.shape
  #print(r)
  data=[]
  new_data=pd.DataFrame(index=np.arange(0,r),columns=['Wilderness_Area', 'Soil_Type','Cover_Type'])
  for i in range(0,r):
      p=0;
      q=0;
      for j in range(10,14):
          if(df.iloc[i,j]==1):
              p=j-9
              break
      for k in range(14,50):
          if(df.iloc[i,k]==1):
              q=k-13
              break
      #new_data.iloc[i]=[p,q]
  #data_out=df.iloc[:,:10]
  #data_out=np.concatenate((data_out,new_data.iloc[:,:]),axis=1)
      new_data.iloc[i] = [p,q,df.iloc[i, c-1]]
  #return data_out
  return new_data

In [ ]:
newData=pd.DataFrame()
r,c=df.shape
for i in range(0,r,500):
  newData=newData.append(cut_down_col(df[i:i+500]))

In [ ]:
newData

,Wilderness_Area,Soil_Type,Cover_Type
0,1,26,5
1,1,26,5
2,1,10,2
3,1,27,2
4,1,26,5
...,...,...,...
7,3,2,3
8,3,2,3
9,3,2,3
10,3,2,3


In [ ]:
  data_out=df.iloc[:,:10]
  data_out=np.concatenate((data_out,newData.iloc[:,:]),axis=1)
 

In [ ]:
data_out

array([[2596,   51,    3, ...,    1,   26,    5],
       [2590,   56,    2, ...,    1,   26,    5],
       [2804,  139,    9, ...,    1,   10,    2],
       ...,
       [2386,  159,   17, ...,    3,    2,    3],
       [2384,  170,   15, ...,    3,    2,    3],
       [2383,  165,   13, ...,    3,    2,    3]])

In [ ]:
df1=pd.DataFrame(data=data_out[0:,0:])
df1.rename(columns = {0:'Elevation', 1:'Aspect', 2:'Slope',3:'Horizontal_Distance_To_Hydrology',4:'Vertical_Distance_To_Hydrology',
                      5:'Horizontal_Distance_To_Roadways',6:'Hillshade_9am',7:'Hillshade_Noon',8:'Hillshade_3pm',9:'Horizontal_Distance_To_Fire_Points',
                      10:'Wilderness_Area',11:'Soil_Type',12:'Cover_Type'}, inplace = True) 

In [ ]:
df1

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,26,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,26,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,10,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,27,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,3,2,3
581008,2391,152,19,67,12,95,240,237,119,845,3,2,3
581009,2386,159,17,60,7,90,236,241,130,854,3,2,3
581010,2384,170,15,60,5,90,230,245,143,864,3,2,3


In [ ]:
num_cols=['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology',
                      'Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points',
                      'Wilderness_Area','Soil_Type']
for var in num_cols:
  scaler=MinMaxScaler()
  df1[var] = scaler.fit_transform(df1[var].values.reshape(-1,1))

In [ ]:
df1['Cover_Type']=(df1['Cover_Type'].apply(lambda x:x-1)).to_frame()

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 13 columns):
Elevation                             581012 non-null float64
Aspect                                581012 non-null float64
Slope                                 581012 non-null float64
Horizontal_Distance_To_Hydrology      581012 non-null float64
Vertical_Distance_To_Hydrology        581012 non-null float64
Horizontal_Distance_To_Roadways       581012 non-null float64
Hillshade_9am                         581012 non-null float64
Hillshade_Noon                        581012 non-null float64
Hillshade_3pm                         581012 non-null float64
Horizontal_Distance_To_Fire_Points    581012 non-null float64
Wilderness_Area                       581012 non-null float64
Soil_Type                             581012 non-null float64
Cover_Type                            581012 non-null int64
dtypes: float64(12), int64(1)
memory usage: 57.6 MB


BAYESIAN INCREMENTAL LEARNING

In [ ]:
#BNN
config = {
    "num_hidden_layers": 4,
    "num_hidden_units": 145,
    "batch_size": 750,
    "epochs":20,
    "learning_rate": 0.001,
    "num_samples": 20,
    "pi": 0.25,
    "sigma_p": 2.5,
    "sigma_p1": 2.5,
    "sigma_p2": 2.5,
}

In [ ]:
ctx = mx.gpu()

In [ ]:
dataset = df1.values
X = dataset[:,0:12].astype(float)
Y = dataset[:,12]
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2)

In [ ]:
class ForestTrainDataset(Dataset):
  def __init__(self):
    self.len=trainX.shape[0]
    self.x_data=(trainX)
    self.y_data=(trainY)
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  def __len__(self):
    return self.len

In [ ]:
class ForestTestDataset(Dataset):
  def __init__(self):
    self.len=testX.shape[0]
    self.x_data=(testX)
    self.y_data=(testY)
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  def __len__(self):
    return self.len

In [ ]:
batch_size=config['batch_size']
num_inputs = 12
num_outputs = 7
dataset=ForestTrainDataset()
train_data=mx.gluon.data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle='true')

In [ ]:
dataset=ForestTestDataset()
test_data=mx.gluon.data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle='false')

In [ ]:
num_train = sum([batch_size for i in train_data])
num_batches = num_train / batch_size
num_test = sum([batch_size for i in test_data])

In [ ]:
print(num_train)
print(num_batches)
print(num_test)

465000
620.0
116250


In [ ]:
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

In [ ]:
num_layers = config['num_hidden_layers']
# define function for evaluating MLP
def net(X, layer_params):
    layer_input = X
    for i in range(len(layer_params) // 2 - 1):
        h_linear = nd.dot(layer_input.astype(float), layer_params[2*i].astype(float)) + layer_params[2*i + 1].astype(float)
        layer_input = relu(h_linear)
    # last layer without ReLU
    output = nd.dot(layer_input.astype(float), layer_params[-2].astype(float)) + layer_params[-1].astype(float)
    return output


In [ ]:
# define network weight shapes
layer_param_shapes = []
num_hidden = config['num_hidden_units']
print("Begin of defining network weight shapes")
for i in range(num_layers + 1):
    if i == 0: # input layer
        W_shape = (num_inputs, num_hidden)
        b_shape = (num_hidden,)
    elif i == num_layers: # last layer
        W_shape = (num_hidden, num_outputs)
        b_shape = (num_outputs,)
    else: # hidden layers
        W_shape = (num_hidden, num_hidden)
        b_shape = (num_hidden,)
    print("i=%d" % i)
    print("w_shape=")
    print(W_shape)
    print("b_shape=")
    print(b_shape)
    layer_param_shapes.extend([W_shape, b_shape])
print("End of defining network weight shapes")

Begin of defining network weight shapes
i=0
w_shape=
(12, 145)
b_shape=
(145,)
i=1
w_shape=
(145, 145)
b_shape=
(145,)
i=2
w_shape=
(145, 145)
b_shape=
(145,)
i=3
w_shape=
(145, 145)
b_shape=
(145,)
i=4
w_shape=
(145, 7)
b_shape=
(7,)
End of defining network weight shapes


In [ ]:
def log_softmax_likelihood(yhat_linear, y):
    return nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

In [ ]:
def gaussian(x, mu, sigma):
    scaling = 1.0 / nd.sqrt(2.0 * np.pi * (sigma ** 2))
    bell = nd.exp(- (x - mu) ** 2 / (2.0 * sigma ** 2))

    return scaling * bell

def scale_mixture_prior(x):
    sigma_p1 = nd.array([config['sigma_p1']], ctx=ctx)
    sigma_p2 = nd.array([config['sigma_p2']], ctx=ctx)
    pi = config['pi']

    first_gaussian = pi * gaussian(x, 0., sigma_p1)
    second_gaussian = (1 - pi) * gaussian(x, 0., sigma_p2)

    return nd.log(first_gaussian + second_gaussian)

In [ ]:
def combined_loss(output, label_one_hot, params, mus, sigmas, log_prior, log_likelihood):

    # Calculate data likelihood
    log_likelihood_sum = nd.sum(log_likelihood(output.astype(float), label_one_hot.astype(float)))

    # Calculate prior
    log_prior_sum = sum([nd.sum(log_prior(param)) for param in params])

    # Calculate variational posterior
    log_var_posterior_sum = sum([nd.sum(gaussian(params[i], mus[i], sigmas[i])) for i in range(len(params))])

    # Calculate total loss
    return 1.0 / num_batches * (log_var_posterior_sum.astype(float) - log_prior_sum.astype(float)) - log_likelihood_sum.astype(float)

In [ ]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [ ]:
def evaluate_accuracy(data_iterator, net, layer_params):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1, 12))
        label = label.as_in_context(ctx)
        output = net(data, layer_params)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

In [ ]:
weight_scale = .1
rho_offset = -8

# initialize variational parameters; mean and variance for each weight
mus = []
rhos = []
for shape in layer_param_shapes:
    mu = nd.random_normal(shape=shape, ctx=ctx, scale=weight_scale)
    rho = rho_offset + nd.zeros(shape=shape, ctx=ctx)
    mus.append(mu)
    rhos.append(rho)
variational_params = mus + rhos

In [ ]:
for param in variational_params:
    param.attach_grad()

In [ ]:
def sample_epsilons(param_shapes):
    epsilons = [nd.random_normal(shape=shape, loc=0., scale=1.0, ctx=ctx) for shape in param_shapes]
    return epsilons

In [ ]:
def softplus(x):
    return nd.log(1. + nd.exp(x))

def transform_rhos(rhos):
    return [softplus(rho) for rho in rhos]

In [ ]:
def transform_gaussian_samples(mus, sigmas, epsilons):
    samples = []
    for j in range(len(mus)):
        samples.append(mus[j] + sigmas[j] * epsilons[j])
    return samples

In [ ]:
epochs = config['epochs']
learning_rate = config['learning_rate']
smoothing_constant = .1
train_acc = []
test_acc = [] 

start_time = time.time()

for i, (data, label) in enumerate(train_data):
    for e in range(20):
        data = data.as_in_context(ctx).reshape((-1, 12))
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 7)

        with autograd.record():
            # sample epsilons from standard normal
            epsilons = sample_epsilons(layer_param_shapes)
            sigmas = transform_rhos(rhos)
            layer_params = transform_gaussian_samples(mus, sigmas, epsilons)
            output = net(data, layer_params)
            loss = combined_loss(output, label_one_hot, layer_params, mus, sigmas,scale_mixture_prior, log_softmax_likelihood)
            
        # backpropagate for gradient calculation
        loss.backward()
        
        # apply stochastic gradient descent to variational parameters
        SGD(variational_params, learning_rate)

        # calculate moving loss for monitoring convergence
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
end_time = time.time()
        
test_accuracy = evaluate_accuracy(test_data, net, mus)
test_acc.append(np.asscalar(test_accuracy))
print("Loss: %s,Test_acc %s" %
          (moving_loss, test_accuracy))

Loss: 5309.635709649373,Test_acc 0.8103749472905175


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


In [ ]:
print(end_time-start_time)

669.6712508201599


In [ ]:
predlist=[]
lbllist=[]
for i, (data, label) in enumerate(test_data):
  data = data.as_in_context(ctx).reshape((-1, 12))
  label = label.as_in_context(ctx)
  output = net(data, mus)
  predictions = nd.argmax(output, axis=1)
  for x in predictions:
    predlist.append(numpy.asscalar(x.asscalar()))
  for y in label:
    lbllist.append(numpy.asscalar(y.asscalar()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
accuracy = accuracy_score(lbllist, predlist)
print('Accuracy: %f' % accuracy)
precision = precision_score(lbllist, predlist,average='macro')
print('Precision: %f' % precision)
recall = recall_score(lbllist, predlist,average='macro')
print('Recall: %f' % recall)
f1 = f1_score(lbllist, predlist,average='macro')
print('F1 score: %f' % f1)
kappa = cohen_kappa_score(lbllist, predlist)
print('Cohens kappa: %f' % kappa)
conf_mat=confusion_matrix(lbllist, predlist)
print(conf_mat)

Accuracy: 0.817655
Precision: 0.712088
Recall: 0.684557
F1 score: 0.695063
Cohens kappa: 0.705082
[[34070  7080     1     0    85    20   839]
 [ 6723 49027   318     7   439   362    90]
 [   16   546  5833   126   142   529     0]
 [    0     0   139   327     0    34     0]
 [  125  1097    23     0   634    22     0]
 [   29   461  1178    43    35  1661     0]
 [  577   102     0     0     1     0  3462]]


In [ ]:
print(classification_report(lbllist, predlist))

              precision    recall  f1-score   support

         0.0       0.82      0.81      0.81     42095
         1.0       0.84      0.86      0.85     56966
         2.0       0.78      0.81      0.79      7192
         3.0       0.65      0.65      0.65       500
         4.0       0.47      0.33      0.39      1901
         5.0       0.63      0.49      0.55      3407
         6.0       0.79      0.84      0.81      4142

    accuracy                           0.82    116203
   macro avg       0.71      0.68      0.70    116203
weighted avg       0.81      0.82      0.82    116203



USING CREME LIBRARY FOR INCREMENTAL LEARNING

In [ ]:
features = df1.iloc[:,0:12]
class_col = df1.iloc[:,12]
features.head()
class_col.head()

0    4
1    4
2    1
3    1
4    4
Name: Cover_Type, dtype: int64

In [ ]:
model = MLPClassifier(solver='sgd', hidden_layer_sizes=(145,145,145,145), alpha=1e-5,
                        max_iter=150, random_state=0, activation='relu',
                        learning_rate_init=0.2)
dataset = df1.values
trainX, testX, trainY, testY = train_test_split(features,class_col,random_state=20190501,test_size=0.2)

r,c=trainX.shape

for i in range(0,r,750):
  X=trainX.iloc[i:i+750]
  y=trainY.iloc[i:i+750]
  model.partial_fit(X, y, classes=np.unique(trainY))

In [ ]:
y_pred = model.predict(testX)
accuracy = accuracy_score(testY, y_pred)
print('Accuracy: %f' % accuracy)
precision = precision_score(testY, y_pred,average='macro')
print('Precision: %f' % precision)
recall = recall_score(testY, y_pred,average='macro')
print('Recall: %f' % recall)
f1 = f1_score(testY, y_pred,average='macro')
print('F1 score: %f' % f1)
kappa = cohen_kappa_score(testY, y_pred)
print('Cohens kappa: %f' % kappa)
matrix = confusion_matrix(testY, y_pred)
print(matrix)

Accuracy: 0.768810
Precision: 0.738549
Recall: 0.546920
F1 score: 0.565353
Cohens kappa: 0.626457
[[34386  7235     5     0     4     1   939]
 [10821 44519  1068     1   110    61   103]
 [    1   477  6454    42     0    70     0]
 [    0     0   429   132     0     0     0]
 [  180  1214    67     0   392     0     0]
 [   46   884  2136    11     0   381     0]
 [  933    27     0     0     0     0  3074]]


In [ ]:
print(classification_report(testY,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.81      0.77     42570
           1       0.82      0.79      0.80     56683
           2       0.64      0.92      0.75      7044
           3       0.71      0.24      0.35       561
           4       0.77      0.21      0.33      1853
           5       0.74      0.11      0.19      3458
           6       0.75      0.76      0.75      4034

    accuracy                           0.77    116203
   macro avg       0.74      0.55      0.57    116203
weighted avg       0.77      0.77      0.76    116203

